In [2]:
import numpy as np

# --- Step 1: Parameters ---
P_M1_0 = np.array([20000., 0., 2000.])
v_M = 300.
P_FY1_0 = np.array([17800., 0., 1800.])
v_FY1 = 120.
P_Decoy = np.array([0., 0., 0.])
P_T = np.array([0., 200., 5.])
R_C = 10.
v_sink = 3.
g = 9.8
t_drop = 1.5
t_fuse_delay = 3.6
t_detonate = t_drop + t_fuse_delay

# --- Step 2: Trajectories (Corrected) ---
# M1 trajectory (unchanged)
d_M1 = (P_Decoy - P_M1_0) / np.linalg.norm(P_Decoy - P_M1_0)
def get_P_M1(t):
    return P_M1_0 + v_M * t * d_M1

# FY1 trajectory (corrected for constant altitude)
d_FY1_horizontal = (P_Decoy[:2] - P_FY1_0[:2]) / np.linalg.norm(P_Decoy[:2] - P_FY1_0[:2])
d_FY1 = np.array([d_FY1_horizontal[0], d_FY1_horizontal[1], 0.])
v_vec_FY1 = v_FY1 * d_FY1
def get_P_FY1(t):
    return P_FY1_0 + v_vec_FY1 * t

# --- Step 3: Detonation Point (Recalculated) ---
P_drop = get_P_FY1(t_drop)
v_drop = v_vec_FY1
P_detonate = P_drop + v_drop * t_fuse_delay + np.array([0, 0, -0.5 * g * t_fuse_delay**2])

# --- Step 4: Cloud Center Trajectory (Updated) ---
def get_P_C(t):
    if t < t_detonate:
        return None
    return P_detonate + np.array([0, 0, -v_sink * (t - t_detonate)])

# --- Step 5: Simulation ---
def dist_point_to_segment(p, a, b):
    v = b - a
    w = p - a
    c1 = np.dot(w, v)
    if c1 <= 0: return np.linalg.norm(p - a)
    c2 = np.dot(v, v)
    if c2 <= c1: return np.linalg.norm(p - b)
    b_proj = c1 / c2
    pb = a + b_proj * v
    return np.linalg.norm(p - pb)

total_shielding_time = 0.0
dt = 0.01
t_start = t_detonate
t_end = t_detonate + 20.0

for t in np.arange(t_start, t_end, dt):
    p_m1_t = get_P_M1(t)
    p_c_t = get_P_C(t)

    distance = dist_point_to_segment(p_c_t, p_m1_t, P_T)

    if distance < R_C:
        total_shielding_time += dt

# --- Final Result ---
print(f"修正后的起爆点 P_detonate: {P_detonate}")
print(f"修正后的有效遮蔽时长: {total_shielding_time:.4f} s")

# 修正后的起爆点 P_detonate: [17188.       0.        1736.496]
# 修正后的有效遮蔽时长: 2.1400 s


修正后的起爆点 P_detonate: [17188.        0.     1736.496]
修正后的有效遮蔽时长: 1.4300 s
